In [ ]:
knitr::opts_chunk$set(echo = TRUE)

## 1. Preparation

### 1.1 Load Necessary Libraries

In [ ]:
library(tidyverse)  # For data manipulation and visualization
library(vegan)      # For ecological analysis
library(ape)        # For phylogenetic analysis
library(microbiome) # For microbiome data transformations
library(mediation)  # For mediation analysis

### 1.2 Load Input Data

In [ ]:
microbiome_data <- read.table("Input/lld.metaphlan2.species.tsv", header = TRUE)  # Species abundance table
phenotype_data <- read.table("Input/lld.phen.tsv", header = TRUE)                 # Phenotype table

## 2. Preprocessing

The original Metaphlan2 output has long taxonomy names, which are not convenient for analysis. We will clean up the names and make the dataset easier to work with.

In [ ]:
# Shorten taxonomy names to keep only the species names
rownames(microbiome_data) <- str_replace_all(rownames(microbiome_data), "^.*s__", "s__")

# Transpose abundance table: rows are samples and columns are species
microbiome_data <- t(microbiome_data)

# Rescale the table to ensure row sums are equal to 1
microbiome_data <- apply(microbiome_data, 1, function(x) x / sum(x)) %>% t() %>% as.data.frame()

# rowSums(microbiome_data) ## Check if the row sums of 'microbiome_data' are all equal to 1


## 3. Ecological Parameters

For analyzing overall ecological parameters, such as alpha and beta diversity, we use the **unfiltered** abundance table.

### 3.1 Alpha Diversity

In [ ]:
# Calculate Shannon diversity using vegan's function, and count the species number (richness).
alpha_diversity <- data.frame(
  Shannon  = vegan::diversity(microbiome_data, index = "shannon"),
  Richness = rowSums(microbiome_data != 0)
)

# Create directory for results if it doesn't exist
if (!dir.exists("Results")) { dir.create("Results") }
# Save the alpha diversity results
write.table(alpha_diversity, "Results/alpha_diversity_metrics.tsv", sep = "\t", row.names = TRUE, col.names = NA, quote = FALSE)

# Check if there is a difference in gut microbiome Shonnon index between hyperglycemia patients and health controls
wilcox.test(alpha_diversity$Shannon ~ phenotype_data$Hyperglycemia)

# Visualize the between-group differences in Shannon index using a boxplot
ggplot(data = cbind(alpha_diversity, phenotype_data), 
       aes(x = factor(Hyperglycemia, labels = c("Control", "Hyperglycemia")), 
           y = Shannon, 
           fill = factor(Hyperglycemia, labels = c("Control", "Hyperglycemia")),
           color = factor(Hyperglycemia, labels = c("Control", "Hyperglycemia")))) +
  geom_boxplot(alpha = 0.7, outlier.shape = 21, outlier.fill = "white") +
  geom_jitter(width = 0.2, alpha = 0.5) +
  stat_summary(fun = mean, geom = "point", shape = 23, size = 3, fill = "white") +
  labs(x = "Group", 
       y = "Shannon Index",
       title = "Shannon Diversity Index by Hyperglycemia Status") +
  scale_fill_manual(values = c("#4169E1", "#FF6347")) +
  scale_color_manual(values = c("#4169E1", "#FF6347")) +
  theme_minimal() +
  theme(legend.position = "none",
        plot.title = element_text(hjust = 0.5, face = "bold"),
        plot.subtitle = element_text(hjust = 0.5),
        axis.title = element_text(face = "bold"),
        axis.text = element_text(size = 10))

### 3.2 Beta Diversity

Principal coordinates analysis (PCoA) can help us reduce the dimension of microbiome composition data, capture the main information of high-dimension dataset and project them to a few number of new dimensions. The PCoA plot can visually reflect the distance (dissimilarity) between samples.

In [ ]:
# Calculate dissimilarities using Bray-Curtis index
microbiome_dist <- vegdist(microbiome_data, index = "bray")

# Perform PCoA
microbiome_pcoa <- ape::pcoa(microbiome_dist)

# Extract principal coordinates
principal_coords <- data.frame(microbiome_pcoa$vectors)

# PCoA plot: visualize the first 2 principal coordinates
# Calculate the percentage of variance explained by each axis
variance_explained <- microbiome_pcoa$values$Relative_eig * 100

ggplot(principal_coords, aes(Axis.1, Axis.2, color = as.factor(phenotype_data$Hyperglycemia))) +
  geom_point(size = 1, alpha = 0.7) +
  xlab(paste0("PCoA1 (", round(variance_explained[1], 2), "%)")) +
  ylab(paste0("PCoA2 (", round(variance_explained[2], 2), "%)")) +
  scale_color_manual(values = c("#4169E1", "#FF6347"), name = "Hyperglycemia") +
  theme_minimal() +
  theme(
    legend.position = "right",
    axis.title = element_text(face = "bold"),
    legend.title = element_text(face = "bold")
  ) +
  labs(title = "PCoA of Gut Microbiome Composition Based on Bray-Curtis Dissimilarity",
       subtitle = "With 95% confidence ellipses") +
  coord_fixed(ratio = 1) +
  stat_ellipse(aes(group = as.factor(phenotype_data$Hyperglycemia)), 
               type = "t", level = 0.95, linetype = 2)

### 3.3 PERMANOVA

Permutational multivariate analysis of variance (PERMANOVA) can help us estimate the proportion of variance explained by different factors.

In [ ]:
permanova_results <- adonis2(microbiome_dist ~ Age + Sex + FruitIntakeFrequency + Glucose, phenotype_data, permutations = 999)

# Save the PERMANOVA results
if (!dir.exists("Results")) { dir.create("Results") }
write.table(as.data.frame(permanova_results), "Results/PERMANOVA_results.tsv", sep = "\t", row.names = TRUE, col.names = NA, quote = FALSE)

## 4. Preparing for Association Analysis

### 4.1 Filtering

Filter species based on mean abundance and prevalence to retain those that are highly prevalent and abundant. This step should be done **before CLR transformation**.

In [ ]:
# Calculate mean abundance and prevalence for each species
mean_abundance <- colMeans(microbiome_data)
prevalence <- colSums(microbiome_data > 0) / nrow(microbiome_data)

# Filter species based on abundance and prevalence criteria. Here we keep species that have a mean abundance greater than 0.01% and are present in more than 5% of samples.
species_to_keep <- colnames(microbiome_data)[mean_abundance > 0.0001 & prevalence > 0.05]
microbiome_filtered <- microbiome_data[, species_to_keep]

# Rescale the filtered table (🤔Think: Is this necessary?)
microbiome_filtered <- apply(microbiome_filtered, 1, function(x) x / sum(x)) %>% t() %>% as.data.frame()

### 4.2 CLR Transformation

Now we use centered log-ratio (CLR) transformation to remove compositional dependency between variables.

In [ ]:
# CLR transformation
# Note that the input table of microbiome::transform() function requires samples in columns and taxa in rows, this is why we transpose the table with the function t() and then transpose it back.
microbiome_clr <- microbiome::transform(t(microbiome_filtered), transform = "clr") %>% t()

### 4.3 Sequencing Depth Adjustment

In metagenomic sequencing-based microbiome studies, sequencing depth often varies across samples, which can introduce bias in diversity estimation and species associations. Two main strategies can address this issue: 

1. **Rarefaction or Downsampling**: This method involves randomly re-sampling the reads from each sample to a uniform read count, ensuring all samples have the same sequencing depth. However, this approach has the downside of discarding valuable data, leading to inefficiencies in both cost and information.

2. **Incorporating Sequencing Depth (Read Count) as a Covariate**: This approach accounts for sequencing depth in downstream statistical analyses, helping to remove its confounding effects. We recommend this method, as it preserves all data and provides a more robust analysis.

Here we use the second strategy to evaluate how sequencing depth influences microbiome diversity estimates and species abundance.

In [ ]:
# Correlation between read count and Shannon diversity / Richness
cor.test(phenotype_data$CleanReadCount, alpha_diversity$Shannon, method = "spearman")
cor.test(phenotype_data$CleanReadCount, alpha_diversity$Richness, method = "kendall")

# Correlation between read count and species abundance
species_read_corr <- data.frame(Species = colnames(microbiome_clr), R = NA, P = NA)

# Loop through each species to calculate correlation with read count
for (i in 1:ncol(microbiome_clr)) {
  corr_test <- cor.test(microbiome_clr[, i], phenotype_data$CleanReadCount, method = "spearman")
  species_read_corr$R[i] <- corr_test$estimate  # Store correlation coefficient
  species_read_corr$P[i] <- corr_test$p.value   # Store p-value
}

# Adjust p-values for multiple testing
species_read_corr$FDR <- p.adjust(species_read_corr$P, method = "fdr")
# Sort results by p-value
species_read_corr <- arrange(species_read_corr, P)

# Save correlation results to file
if (!dir.exists("Results")) { dir.create("Results") }
write.table(species_read_corr, "Results/species_abundance_read_count_correlation.tsv", sep = "\t", row.names = FALSE, col.names = TRUE, quote = FALSE)

## 5. Association Analysis

### 5.1 Association Analysis between Microbial Species Abundance and Phenotypes (Unadjusted)

We use **Spearman's correlation** for analyzing the association between two continuous variables, and the **Wilcoxon rank-sum test** for the association between a continuous variable and a binary variable (0/1), when no confounding effects are considered.

In [ ]:
# Association between species abundance and binary variable (hyperglycemia)
hyperglycemia_wilcox <- data.frame(Species = colnames(microbiome_clr), Mean_0 = NA, Mean_1 = NA, P = NA)

# Loop through each species in the microbiome_clr dataset
for (i in 1:ncol(microbiome_clr)) {
  # Perform Wilcoxon rank-sum test for each species
  test_result <- wilcox.test(microbiome_clr[, i] ~ phenotype_data$Hyperglycemia)
  # Calculate mean abundance for non-hyperglycemic group (0)
  hyperglycemia_wilcox$Mean_0[i] <- mean(microbiome_clr[phenotype_data$Hyperglycemia == 0, i])
  # Calculate mean abundance for hyperglycemic group (1)
  hyperglycemia_wilcox$Mean_1[i] <- mean(microbiome_clr[phenotype_data$Hyperglycemia == 1, i])
  # Store the p-value from the Wilcoxon test
  hyperglycemia_wilcox$P[i] <- test_result$p.value
}

# Calculate the difference in mean abundance between hyperglycemic and non-hyperglycemic groups
hyperglycemia_wilcox$Difference <- hyperglycemia_wilcox$Mean_1 - hyperglycemia_wilcox$Mean_0
# Adjust p-values for multiple testing using FDR method
hyperglycemia_wilcox$FDR <- p.adjust(hyperglycemia_wilcox$P, method = "fdr")
# Sort the results by p-value
hyperglycemia_wilcox <- arrange(hyperglycemia_wilcox, P)

# Save results of Wilcoxon test for hyperglycemia
if (!dir.exists("Results")) { dir.create("Results") }
write.table(hyperglycemia_wilcox, "Results/hyperglycemia_species_association_wilcoxon.tsv", sep = "\t", row.names = FALSE, col.names = TRUE, quote = FALSE)

# Association between species abundance and continuous variable (glucose)
glucose_spearman <- data.frame(Species = colnames(microbiome_clr), R = NA, P = NA)

# Perform Spearman correlation for each species
for (i in 1:ncol(microbiome_clr)) {
  test_result <- cor.test(microbiome_clr[, i], phenotype_data$Glucose, method = "spearman")
  glucose_spearman$R[i] <- test_result$estimate  # Correlation coefficient
  glucose_spearman$P[i] <- test_result$p.value   # P-value
}

# Adjust p-values for multiple testing
glucose_spearman$FDR <- p.adjust(glucose_spearman$P, method = "fdr")
# Sort results by p-value
glucose_spearman <- arrange(glucose_spearman, P)

# Save results of Spearman correlation for glucose
if (!dir.exists("Results")) { dir.create("Results") }
write.table(glucose_spearman, "Results/glucose_species_association_spearman.tsv", sep = "\t", row.names = FALSE, col.names = TRUE, quote = FALSE)

### 5.2 Association Analysis between Microbial Species Abundance and Phenotypes (Adjusting for Covariates)

For association analyses involving potential confounders, we apply **linear regression** when the outcome variable (y) is continuous, and **logistic regression** when the outcome is binary (0/1). To control for confounding effects, we include relevant covariates in the models. In microbiome studies, **sequencing depth** is often included as a covariate. Additionally, in human studies, **age** and **sex** are commonly considered important confounding factors.

In [ ]:
# Association between species abundance and binary variables
hyperglycemia_reg <- data.frame(Species = colnames(microbiome_clr), Beta = NA, P = NA)

# Loop through each species and perform logistic regression
for (i in 1:ncol(microbiome_clr)) {
  model <- glm(Hyperglycemia ~ Age + Sex + CleanReadCount + microbiome_clr[, i], data = phenotype_data, family = "binomial")
  model_summary <- summary(model)
  hyperglycemia_reg$Beta[i] <- model_summary$coefficients[5, 1]  # Extract Beta coefficient
  hyperglycemia_reg$P[i] <- model_summary$coefficients[5, 4]     # Extract P-value
}

# Adjust for multiple testing and sort results
hyperglycemia_reg$FDR <- p.adjust(hyperglycemia_reg$P, method = "fdr")
hyperglycemia_reg <- arrange(hyperglycemia_reg, P)

# Save results to file
if (!dir.exists("Results")) { dir.create("Results") }
write.table(hyperglycemia_reg, "Results/hyperglycemia_species_association_logistic_regression.tsv", sep = "\t", row.names = FALSE, col.names = TRUE, quote = FALSE)

# Association between species abundance and continuous variables
glucose_reg <- data.frame(Species = colnames(microbiome_clr), Beta = NA, P = NA)

# Loop through each species and perform linear regression
for (i in 1:ncol(microbiome_clr)) {
  model <- lm(Glucose ~ Age + Sex + CleanReadCount + microbiome_clr[, i], data = phenotype_data)
  model_summary <- summary(model)
  glucose_reg$Beta[i] <- model_summary$coefficients[5, 1]  # Extract Beta coefficient
  glucose_reg$P[i] <- model_summary$coefficients[5, 4]     # Extract P-value
}

# Adjust for multiple testing and sort results
glucose_reg$FDR <- p.adjust(glucose_reg$P, method = "fdr")
glucose_reg <- arrange(glucose_reg, P)

# Save results to file
if (!dir.exists("Results")) { dir.create("Results") }
write.table(glucose_reg, "Results/glucose_species_association_linear_regression.tsv", sep = "\t", row.names = FALSE, col.names = TRUE, quote = FALSE)

## 6. Mediation Analysis

**Mediation analysis** is a statistical technique that helps us infer causal or regulatory relationships among multiple factors. In our previous results, we observed that the bacterial species *Eubacterium eligens* showed the most significant negative association with glucose levels. We are interested in exploring whether *E. eligens* could potentially be used as a target to decrease glucose levels through lifestyle interventions, such as increased fruit consumption.

To investigate this, we can use mediation analysis to test the regulatory relationship between three key variables:

1. Independent variable: Fruit intake frequency
2. Mediator: Abundance of *Eubacterium eligens*
3. Dependent variable: Glucose levels

This analysis will help us understand whether the effect of fruit intake on glucose levels is mediated by changes in the abundance of *E. eligens* in the gut microbiome.

In [ ]:
Microbe <- microbiome_clr[, grep("s__Eubacterium_eligens", colnames(microbiome_clr))]
mediation_input <- data.frame(phenotype_data, Microbe)

# Test relationship between independent variable (FruitIntakeFrequency) and mediator (Microbe)
fit_mediator <- lm(Microbe ~ Age + Sex + CleanReadCount + FruitIntakeFrequency, data = mediation_input)
summary(fit_mediator)

# Test relationship between mediator (Microbe) and outcome (Glucose)
fit_dv <- lm(Glucose ~ Age + Sex + CleanReadCount + FruitIntakeFrequency + Microbe, data = mediation_input)
summary(fit_dv)

# Mediation analysis
mediation_res <- mediate(fit_mediator, fit_dv, treat = 'FruitIntakeFrequency', mediator = 'Microbe', boot = TRUE)
summary(mediation_res)

# ACME represents the indirect effect of FruitIntakeFrequency on glucose via *Eubacterium eligens*.
# ADE represents the direct effect of FruitIntakeFrequency on glucose.
# Total Effect is the sum of direct and indirect effects.
# Prop. Mediated describes the proportion of the effect that is mediated by *Eubacterium eligens*.